<a href="https://colab.research.google.com/github/OrionXV/ISROProject/blob/main/plotfitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math 
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt 
import numpy as np 
from scipy.optimize import leastsq
from scipy.special import erf
%config InLineBackend.figure_format = 'retina'

def z_func(B, C, D):
    return (2*B + (C**2)*D)/(2*C)

def objective_func(T, A, B, C, D, E, F):
    #return A*T**4 + B*T**3 + C*T**2 + D*T + E
    return math.sqrt(np.pi*0.5)*A*C*np.exp(D/2*((2*B)+(C**2)*(D/2)-(2*(T*E+ F))))*(erf(z_func(B, C, D))- erf(z_func(B, C, D) - (T*E+ F)/C))
    #return A*np.exp(-(T-B)**2/(2*C**2))+np.exp(-D*T)
    
def objective_cost_func(params, x, y):
    a0, b0, c0, d0, e0, f0 = params[0], params[1], params[2], params[3], params[4], params[5]
    return np.sqrt(((y - objective_func(x, a0, b0, c0, d0, e0, f0))**2)/y)
    


def min_max_scaler(df):
    df['RATESCALED'] = (df['RATE']- df['RATE'].min())/(df['RATE'].max()- df['RATE'].min())
    df['TIMESCALED'] = (df['TIME']- df['TIME'].min())/(df['TIME'].max()- df['TIME'].min())*2

def rev_scaler(df):
    df['TRUEFIT'] = df['FIT']*(df['RATE'].max()- df['RATE'].min()) + df['RATE'].min()

In [ ]:
from scipy.signal import find_peak

def tolerence_func(r, mid, start, stop):
    if mid - start < 0:
        start = 0
    else:
        start = mid - start
    if mid + stop > r:
        stop = r
    else:
        stop = mid + stop
    return start, stop

def peak_fitter(df, t1 = 500, t2 = 750):
    peak_list, _ = find_peaks(df['RATE'], distance= df['RATE'].median()*1.005)
    min_max_scaler(df)
    for peak in peak_list:
        start, stop = tolerence_func(len(df), df[df['RATE'] == peak].index.values, t1, t2)
        TIME = df['TIMESCALED'][start:stop]
        RATE = df['RATESCALED'][start:stop]

        params = [0.2, 0.3, 0.3, 0.4, 0.1, 0.1]

        resultsq = leastsq(objective_cost_func, params, (TIME, RATE))
        a, b, c, d, e, f = resultsq[0][0], resultsq[0][1], resultsq[0][2], resultsq[0][3], resultsq[0][4], resultsq[0][5]


        new_curve = objective_func(TIME, a, b, c, d, e, f)

        df['FIT'][start:stop] = new_curve
        rev_scaler(df)
    
    plt.plot(df['TIME'], df['TRUEFIT'])
